In [133]:
import json
with open('../data/duckie_sim_train.json') as json_file:
    sim_data = json.load(json_file)
with open('../data/duckie_js_train.json') as json_file:
    js_data = json.load(json_file)

In [124]:
!mkdir -p merged_dataset
!mkdir -p merged_dataset/rgb

In [134]:
sim_images = sim_data["images"]

In [135]:
js_images = js_data["images"]

In [136]:
len(js_images)

9000

In [137]:
import shutil
from tqdm import tqdm
for image in tqdm(js_images):
    filename = image["file_name"]
    shutil.copy(f"../data/dt_segmentation_dataset/new/wo_bezier/rgb_orig/{filename}", f"merged_dataset/rgb/js_{filename}")
    

100%|██████████| 9000/9000 [00:05<00:00, 1654.33it/s]


In [138]:
for image in tqdm(sim_images):
    filename = image["file_name"]
    shutil.copy(f"../data/dt_segmentation_dataset/sim/rgb_160_120//{filename}", f"merged_dataset/rgb/sim_{filename}")
    

100%|██████████| 50000/50000 [00:09<00:00, 5412.31it/s] 


In [139]:
#Fresh data
with open('../data/duckie_sim_train.json') as json_file:
    sim_data = json.load(json_file)
with open('../data/duckie_js_train.json') as json_file:
    js_data = json.load(json_file)
    


In [149]:
merged_images=[]
merged_id = 0
js_image_mapping = {}
sim_image_mapping = {}
for image in js_data["images"]:
    new_image = image.copy()
    new_image["file_name"] = "js_" + image["file_name"]
    js_image_mapping[image["id"]] = merged_id
    new_image["id"] = merged_id
    merged_id+=1
    merged_images.append(new_image)


In [150]:
merged_images[0]

{'file_name': 'js_8313.jpg', 'height': 120, 'width': 160, 'id': 0}

In [142]:
for image in sim_data["images"]:
    new_image = image.copy()
    new_image["file_name"] = "sim_" + image["file_name"]
    #print(image["id"])
    sim_image_mapping[image["id"]] = merged_id
    new_image["id"] = merged_id
    merged_id+=1
    merged_images.append(new_image)

In [143]:
len(merged_images), len(sim_images), len(js_images)

(59000, 50000, 9000)

In [144]:
merged_annotations = []

In [145]:
#js_data["annotations"][0]

In [146]:
annotation_id = 0
for annotation in js_data["annotations"]:
    new_annotation = annotation.copy()
    new_annotation["image_id"] = js_image_mapping[annotation["image_id"]]
    new_annotation["id"] = annotation_id
    annotation_id+=1
    merged_annotations.append(new_annotation)



In [147]:
for annotation in sim_data["annotations"]:
    new_annotation = annotation.copy()
    new_annotation["image_id"] = sim_image_mapping[annotation["image_id"]]
    new_annotation["id"] = annotation_id
    annotation_id+=1
    merged_annotations.append(new_annotation)
    

In [148]:
data={"categories":sim_data["categories"], "images":merged_images,"annotations":merged_annotations}
with open(f"../data/duckie_merged_train.json","w") as f:
    json.dump(data,f)

In [151]:
import glob

In [155]:
files = glob.glob("merged_dataset/rgb/*.jpg")
import shutil, os

In [157]:
for file in tqdm(files):
    basename = os.path.basename(file)
    shutil.copy(file, f"../data/dt_segmentation_dataset/merged/rgb/{basename}")
    

100%|██████████| 68093/68093 [00:04<00:00, 13933.11it/s]
